In [ ]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    # Convert "time string" to "timestamp" in seconds
    df['start_sec'] = df['start_time'].apply(lambda x: int(x[:2])*3600 + int(x[3:5])*60 + int(x[6:8]))
    df['end_sec'] = df['end_time'].apply(lambda x: int(x[:2])*3600 + int(x[3:5])*60 + int(x[6:8]))
    return df

In [ ]:
try:
    # Load two labeled datasets from Google Drive
    # felix = load_data('/content/drive/MyDrive/Vent - 12_27_24 - Day 1.csv')
    # alex = load_data('/content/drive/MyDrive/Vent - 12_27_24 - Day 1 - Alex.csv')
    felix = load_data('/content/drive/MyDrive/Vent - 12_27_24 - Day 1 2of2.csv')
    alex = load_data('/content/drive/MyDrive/Vent - 12_27_24 - Day 1 2of2 - Alex.csv')

    # Alex's dataset is shorter than Felix's

    # Find the end time (in seconds) of the last breath in Alex's data
    alex_end_sec = alex.iloc[-1]['end_sec']

    # Trim Felix's data to only include entries that start before Alex's last breath ends
    felix_overlap = felix[felix['start_sec'] < alex_end_sec]

    # If the number of breaths doesn't match, warn and truncate to match Alex's length
    if len(felix_overlap) != len(alex):
        print(f"Warning: Mismatched number of events (Felix={len(felix_overlap)} vs Alex={len(alex)}). Truncating Felix's to match Alex's.")
        felix_overlap = felix_overlap.head(len(alex))

    # Extract label lists
    labels_felix = felix_overlap['label'].tolist()
    labels_alex = alex['label'].tolist()

    # Compute Cohen's Kappa
    kappa = cohen_kappa_score(labels_alex, labels_felix)
    print(f"Cohen's Kappa = {kappa:.4f}")
    print("Reliability:")
    if kappa < 0:
        print(" Worse than random agreement")
    elif kappa < 0.2:
        print(" Slight agreement")
    elif kappa < 0.4:
        print(" Fair agreement")
    elif kappa < 0.6:
        print(" Moderate agreement")
    elif kappa < 0.8:
        print(" Substantial agreement")
    else:
        print(" Almost perfect agreement")

except Exception as e:
    print("Error:", e)

Cohen's Kappa = 0.9818
Reliability:
 Almost perfect agreement
